In [1]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# and batch size
INIT_LR = 1e-4
EPOCHS = 20
BS = 32

In [3]:
DIRECTORY = "/content/dataset/Data_set"
CATEGORIES = ["with_mask","without_mask"]

data = []
labels = []

for category in CATEGORIES:
  path = os.path.join(DIRECTORY,category)
  for img in os.listdir(path):
    img_path = os.path.join(path,img)
    image = load_img(img_path,target_size=(224,224))
    image = img_to_array(image)
    image = preprocess_input(image)

    data.append(image)
    labels.append(category)

FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset/Data_set/with_mask'

In [ ]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels= to_categorical(labels)


In [ ]:
data = np.array(data, dtype='float32')
labels = np.array(labels)

In [4]:
trainX,testX,trainy,testy = train_test_split(data,labels,test_size=0.2,stratify = labels,random_state=42)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
trainy

In [ ]:
aug = ImageDataGenerator(
    rotation_range = 20,
    zoom_range = 0.15,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.15,
    horizontal_flip = True,
    fill_mode = 'nearest'

)

In [ ]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False


/tmp/ipython-input-1972609701.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  baseModel = MobileNetV2(weights="imagenet", include_top=False,


In [ ]:
from tensorflow.keras.optimizers import Adam

# simple fixed learning rate
opt = Adam(learning_rate=INIT_LR)

# compile
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
print("[INFO] training head...")


[INFO] training head...


In [ ]:
history = model.fit(
	aug.flow(trainX, trainy, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testy),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - accuracy: 0.7198 - loss: 0.6441 - val_accuracy: 0.9842 - val_loss: 0.2535
Epoch 2/20
 1/47 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 1.0000 - loss: 0.3142

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


47/47 ━━━━━━━━━━━━━━━━━━━━ 15s 307ms/step - accuracy: 1.0000 - loss: 0.3142 - val_accuracy: 0.9842 - val_loss: 0.2483
Epoch 3/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.9479 - loss: 0.2776 - val_accuracy: 0.9894 - val_loss: 0.1282
Epoch 4/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 16s 308ms/step - accuracy: 0.9375 - loss: 0.1803 - val_accuracy: 0.9894 - val_loss: 0.1273
Epoch 5/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 89s 2s/step - accuracy: 0.9661 - loss: 0.1652 - val_accuracy: 0.9894 - val_loss: 0.0843
Epoch 6/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 15s 295ms/step - accuracy: 0.9375 - loss: 0.1619 - val_accuracy: 0.9894 - val_loss: 0.0838
Epoch 7/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 91s 2s/step - accuracy: 0.9827 - loss: 0.1062 - val_accuracy: 0.9894 - val_loss: 0.0652
Epoch 8/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 15s 297ms/step - accuracy: 0.9688 - loss: 0.1118 - val_accuracy: 0.9894 - val_loss: 0.0648
Epoch 9/20
47/47 ━━━━━━━━━━━━━━━━━━━━ 122s 2s/step - accuracy: 0.9846 - loss: 0.0871 - val_accuracy: 0.9894 - val_los

In [ ]:
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

[INFO] evaluating network...
12/12 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step


In [ ]:
predIdxs = np.argmax(predIdxs, axis=1)

In [ ]:
print(classification_report(testy.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.98      0.99      0.99       200
without_mask       0.99      0.98      0.99       179

    accuracy                           0.99       379
   macro avg       0.99      0.99      0.99       379
weighted avg       0.99      0.99      0.99       379



In [ ]:
print("[INFO] saving mask detector model...")
model.save("mask_detector.h5")

[INFO] saving mask detector model...
